# Create Monthly CESM ice flux files

In [30]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import matplotlib

%matplotlib inline

##### Load files

In [31]:
mask  = nc.Dataset('/ocean/brogalla/GEOTRACES/ariane_runs/ANHA12_Ariane_mesh.nc')
tmask = np.array(mask.variables['tmask'])
tmask = np.array(tmask[0,:,:,:])

mesh  = nc.Dataset('/data/brogalla/old/meshmasks/ANHA12_mesh1.nc')
mlons = np.array(mesh.variables['nav_lon'])
mlats = np.array(mesh.variables['nav_lat'])
mlons = np.array(mlons)
mlats = np.array(mlats)

cond = (tmask > 0.1) 
Z_masked = np.ma.masked_where(cond, tmask) 

In [32]:
folder = '/ocean/brogalla/GEOTRACES/data/NCAR/'

faero_001_n = nc.Dataset(folder+'merged_faero_ocn001_nh.nc')  #hydrophilic black carbon
faero_002_n = nc.Dataset(folder+'merged_faero_ocn002_nh.nc')  #hydrophobic black carbon
faero_003_n = nc.Dataset(folder+'merged_faero_ocn003_nh.nc') #dust

ocn001 = np.array(faero_001_n.variables['faero_ocn001'])
ocn002 = np.array(faero_002_n.variables['faero_ocn002'])
ocn003 = np.array(faero_003_n.variables['faero_ocn003'])
time   = faero_001_n.variables['time']
lons   = np.array(faero_001_n.variables['TLON'])
lats   = np.array(faero_001_n.variables['TLAT'])

In [33]:
for i in range(0,lons.shape[0]):
    for j in range(0,lons.shape[1]):
        if lons[i,j] >= 180:
            lons[i,j] = -360+lons[i,j]

##### Functions

In [34]:
def save_file(filename, field1, field2, field3):
    ncd = nc.Dataset(filename, 'w', zlib=True)
    
    ncd.createDimension('x',len(mesh.dimensions['x']))
    ncd.createDimension('y',len(mesh.dimensions['y']))
    ncd.createDimension('time_counter',None)
    
    # variables
    dust             = ncd.createVariable('dust', 'float64', ('y','x'))
    dust.units       = 'kg/m2 s'
    dust.long_name   = 'Dust deposition flux'  
    dust.coordinates = 'nav_lon nav_lat'
    dust[:]          = field1[:]
    
    bc_philic             = ncd.createVariable('bc_philic', 'float64', ('y','x'))
    bc_philic.units       = 'kg/m2 s'
    bc_philic.long_name   = 'Hydrophilic black carbon'  
    bc_philic.coordinates = 'nav_lon nav_lat'
    bc_philic[:]          = field2[:]
    
    bc_phobic             = ncd.createVariable('bc_phobic', 'float64', ('y','x'))
    bc_phobic.units       = 'kg/m2 s'
    bc_phobic.long_name   = 'Hydrophobic black carbon'  
    bc_phobic.coordinates = 'nav_lon nav_lat'
    bc_phobic[:]          = field3[:]
    
    print('saved ', filename)

    ncd.close()
    return

In [35]:
def interp_np(nav_lon, nav_lat, var_in, lon_ANHA12, lat_ANHA12):
    ''' Interpolate some field to ANHA12 grid.
        The function is based on the bilinear interpolation in scipy, griddata 
        =======================================================================
            nav_lon, nav_lat        : input field lon/lat
            lon_ANHA12, lat_ANHA12  : ANHA12 defined lons/lats
            var_in                  : 2-D model variable
    '''
    from scipy.interpolate import griddata
    LatLonPair = (nav_lon, nav_lat)
    var_out = griddata(LatLonPair, var_in, (lon_ANHA12, lat_ANHA12), method='linear')
    # Take nearest neighbour interpolation to fill nans
    var_fill = griddata(LatLonPair, var_in, (lon_ANHA12, lat_ANHA12), method='nearest')
    
    var_out[np.isnan(var_out)] = var_fill[np.isnan(var_out)]
    return var_out

In [36]:
def find_dates(file_year, time):
    file_date_start = file_year*365
    file_date_end = (file_year+1)*365
    
    start_index = []
    end_index = []

    for i in range(0,len(time)):
        if time[i] == file_date_start:
            start_index = i
        elif time[i] == file_date_end:
            end_index = i

#     print('start index: ', start_index)
#     print('end index: ', end_index)
    
    return start_index, end_index

In [37]:
def data_to_ANHA12(file_year, time, lons, lats, ocn001_masked, ocn002_masked, ocn003_masked, savefiles=False):
    
    start_index, end_index = find_dates(file_year, time)
    
    interp_dst = np.empty((12, 2400, 1632))
    interp_bc1  = np.empty((12, 2400, 1632))
    interp_bc2  = np.empty((12, 2400, 1632))

    # loop over the months:
    for i in range(0,12):
        filt_ocn001 = ocn001_masked[start_index+i,:,:][~ocn001_masked[start_index+i,:,:].mask].data
        filt_ocn002 = ocn002_masked[start_index+i,:,:][~ocn002_masked[start_index+i,:,:].mask].data
        filt_ocn003 = ocn003_masked[start_index+i,:,:][~ocn003_masked[start_index+i,:,:].mask].data
        filt_lons1 = lons[~ocn001_masked[start_index+i,:,:].mask].data
        filt_lons2 = lons[~ocn002_masked[start_index+i,:,:].mask].data
        filt_lons3 = lons[~ocn003_masked[start_index+i,:,:].mask].data
        filt_lats1 = lats[~ocn001_masked[start_index+i,:,:].mask].data
        filt_lats2 = lats[~ocn002_masked[start_index+i,:,:].mask].data
        filt_lats3 = lats[~ocn003_masked[start_index+i,:,:].mask].data

        interp_dst[i,:,:] = interp_np(filt_lons3, filt_lats3, filt_ocn003, mlons, mlats)
        interp_bc1[i,:,:]  = interp_np(filt_lons1, filt_lats1, filt_ocn001, mlons, mlats)
        interp_bc2[i,:,:]  = interp_np(filt_lons2, filt_lats2, filt_ocn002, mlons, mlats)
        
    if savefiles:
        location='/ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/'
        
        for i in range(1,13):
            if i < 10:
                save_file(location+'ice_flux_y'+str(file_year)+'m0'+str(i)+'.nc',interp_dst[i-1,:,:], \
                         interp_bc1[i-1,:,:], interp_bc2[i-1,:,:])
            else:
                save_file(location+'ice_flux_y'+str(file_year)+'m'+str(i)+'.nc',interp_dst[i-1,:,:], \
                         interp_bc1[i-1,:,:], interp_bc2[i-1,:,:])
        
    
    return interp_dst, interp_bc1, interp_bc2

##### Calculations

In [38]:
ocn001_masked = np.ma.masked_where((ocn001 >= 1e30), ocn001) 
ocn002_masked = np.ma.masked_where((ocn002 >= 1e30), ocn002) 
ocn003_masked = np.ma.masked_where((ocn003 >= 1e30), ocn003) 

In [39]:
for year in np.arange(2002,2003,1):
    print(year)
    interp_dst, interp_bc1, interp_bc2 = data_to_ANHA12(year, time, lons, lats, \
                                                        ocn001_masked, ocn002_masked, ocn003_masked,\
                                                        savefiles=True)

2002
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/ice_flux_y2002m01.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/ice_flux_y2002m02.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/ice_flux_y2002m03.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/ice_flux_y2002m04.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/ice_flux_y2002m05.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/ice_flux_y2002m06.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/ice_flux_y2002m07.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/ice_flux_y2002m08.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/ice_flux_y2002m09.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/ice_flux_y2002m10.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/ice_